# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** — ответить на вопросы заказчика:
- Есть ли зависимость между количеством детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

**Иследование** пройдёт в несколько этапов:
- Обзор данных
- Предобработка данных
- Анализ данных
- Интерпретация результатов и выводы

## Шаг 1. Обзор данных

In [1]:
import pandas as pd # импорт библиотеки Pandas

In [2]:
try:
    data = pd.read_csv('datasets\data.csv')  #чтение файла и просмотр первых 15 строк
except:
    data = pd.read_csv('/datasets/data.csv')
    
display(data.head(15))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
#код ревьюера
data = pd.read_csv('/datasets/data.csv')

**Документация к данным:**
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

По первому впечатлению - некоторые стобцы дают одну и ту же информацию, например, столбцы `'education'` и `'education_id'`, `'family_status'` и `'family_status_id'` Данные о стаже работе выражены отрицательными числами. Стобец `'total_income'`- не уточняется, в какой валюте измеряется. Нужно вытащить более детальную информацию о датафрейме.

In [4]:
data.info() # просмотр общей информации о  данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В столбцах ежемесячного дохода и трудового стажа есть пропуски, причем одинаковое количество, возможно что в одних и тех же строках. Конечно, есть вероятность, что в банк обращались люди и без стажа, и без дохода, нужно будет проанализировать пропуски по типу занятости и при необходимости заполнить их медианным значением.

In [5]:
data.describe() 
# проверим числовые столбцы, так как в одном были замечены отрицательные числа: 
# может, по минимуму и максимуму мы увидим аномалии

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Есть интересные находки: максимум детей 20, минимальный возраст 0, отрицательный трудовой стаж и его заоблачные максимальные показатели. Возможно, выгрузка отрицательных показателей связана с технической ошибкой. Прежде чем перейти к ответам на вопросы, план по предобработке выходит примерно таким:
* устранить пропуски при необходимости медианным значением - так оно будет более корректно соответствовать действительности, а еще медиана мало подвержена влиянию артефактов в данных.
* пройтись по каждому столбцу, проверить уникальные значения, устранить аномалии;
* убрать отрицательные показатели в трудовом стаже, перевести в стаж в года для удобства;
* доход и стаж привести к целочисленным значениям: лишние тысячные доли копеек и дней только мешают восприятию информации;
* удалить дубликаты.


## Шаг 2. Устраняем пропуски

In [6]:
print(data.isna().sum()) # посмотрим общее число пропусков 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


In [7]:
# отсортируем датафрейм по наличию пропусков в обоих пунктах и посмотрим, в каких типах занятости они представлены.

print(data[(data['days_employed'].isna() == True) & (data['total_income'].isna() == True)]['income_type'].value_counts())

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64


Данные не были указаны больше всего у работающих людей, их доход явно больше нуля. Следовательно, будем заполнять пропуски медианными значениями.

In [8]:
data['days_employed'] = data['days_employed'].abs() #переводим столбец в положительные значения

# сгруппируем по типу занятости и к новому столбцу применим функцию apply(),в условии которой заменим нули на медианное значение каждой группы
data['years_employed'] = data.groupby('income_type')['days_employed'].apply(lambda x: x.fillna(x.median()))
data['years_employed'] = (data['years_employed'] / 365).astype(int) # дни переведем в года для удобства восприятия

display(data.head(10)) # самопроверка

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,932
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,2
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,7
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,0
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,18
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,5


Функция работает, отлично! Со столбцом 'total_income' процесс пойдет быстрее.

In [9]:
# проделаем те же действия со столбцом дохода, новый столбец назовем month_income - ежемесячный доход, так будет логичнее.
data['month_income'] = data.groupby('income_type')['total_income'].apply(lambda x: x.fillna(x.median())).astype(int)

data = data.drop(columns=['days_employed', 'total_income'], axis=1) # удалим старые и ненужные столбцы 

data.isna().sum() # проверка - все ли пропуски ушли

children            0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
purpose             0
years_employed      0
month_income        0
dtype: int64

С пропусками покончено! Каждая группа занятых получила свое медианное значение стажа работы и ежемесячного дохода. Попутно стало заметно, что, например, у пенсионеров стаж работы почти 1000 лет, причем это не единичный случай, иначе бы медианное значение не было таким высоким, оно устойчиво к выбросам. Либо данные всех пенсионеров заполняли с ошибками, либо был какой-то сбой в выгрузке данных.



## Шаг 3. Ищем дубликаты

Пробежимся по остальным столбцам, исправим ошибки, уберем лишние значения и, наконец, отшлифуем наш датасет для дальнейшей работы.

Для анализа числовых столбцов, содержащих категориальные данные, хорошо подойдет метод value_counts() - он выводит уникальные значения столбца и подсчитывает частоту встречаемости; если есть какие-то несоответствующие значения, мы сможем оперативно их устранить.
Для столбцов, содержащих строковые значения, этот же метод поможет отследить неявные дубликаты - например, один и тот же показатель может быть написан в разном регистре. Такие дубликаты метод duplicated() отследить не сможет. 

In [10]:
data['children'].value_counts() #уникальные значения столбца 'children'

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

47 строк со значением "-1" - тут явно ошибка в заполнении. Сомнения вызывают двадцать детей, если бы это было единичное значение, но их 76, выглядит слишком нереалистично... Будем считать, что имелось в виду два ребенка, просто кто-то нажал лишний ноль.

In [11]:
data['children'] = data['children'].replace({20: 2, # методом replace исправим значение "минус один ребенок" на просто одного, 
                                            -1: 1}) # а двадцать переведем в два
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [12]:
# следующий столбец - 'dob_years', сгруппируем датафрейм по столбцу и выведем количество уникальных значений
print(data.groupby('dob_years')['dob_years'].count())

dob_years
0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64


В датафрейме нулевой возраст целых 101 раз, однако доля таких строк от всего количества 0.4%, что не должно повлиять на будущие расчеты. 

In [13]:
data['education'].value_counts() # уникальные значения столбца 'education'

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

В данных об образовании обнаружились неявные дубликаты: те же названия, но записанные в разных регистрах. Исправим это недоразумение.

In [14]:
data['education'] = data['education'].str.lower() # метод str.lower() приводит все строчные значения к нижнему регистру

In [15]:
print(data.groupby('education').agg({'education_id': 'count',
                                    'education': 'count'}))
# проверим, соответствуют ли значения 'education_id' столбцу образования. Все сходится.

                     education_id  education
education                                   
высшее                       5260       5260
начальное                     282        282
неоконченное высшее           744        744
среднее                     15233      15233
ученая степень                  6          6


In [16]:
# сделаем то же со столбцами о семейном положении - лишних значений нет, идентификаторы соответствуют.
print(data.groupby('family_status').agg({'family_status': 'count',
                                    'family_status_id': 'count'}))

                       family_status  family_status_id
family_status                                         
Не женат / не замужем           2813              2813
в разводе                       1195              1195
вдовец / вдова                   960               960
гражданский брак                4177              4177
женат / замужем                12380             12380


In [17]:
data['gender'].value_counts() # посмотрим, насколько бинарен гендерный статус в датафрейме

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [18]:
# мы уже работали с этим столбцом, но посмотрим подробнее уникальные значения - вроде ничего необычного
data['income_type'].value_counts() 

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [19]:
data['debt'].unique() # уникальные значения столбца debt: все в порядке

array([0, 1])

In [20]:
# самое сложное - нужно вывести уникальные значения столбца с целями кредита, 
# проанализировать их вручную и отсортировать по отдельным категориям.

data['purpose'].value_counts() # вывод уникальных значений

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Здесь бы пригодилась лемматизация, но навыков пока что по ней нет, поэтому вручную анализируем список и составляем список сами... Все цели получения кредита можно сгруппировать на несколько больших категорий:
* свадьба
* автомобиль
* жилье и недвижимость
* образование

In [21]:
# сведем обширные данные из столбца purpose в purpose_category с помощью функции:
# если часть ключевого слова есть в столбце purpose, то функция присвоит ей соответствующую категорию.

def purpose_group(purpose):
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'автомоб' in purpose:
        return 'операции с автомобилем'
    if ('жиль' in purpose) or ('недвижим' in purpose):
        return 'операции с недвижимостью'
    if 'образован' in purpose:
        return 'получение образования'

data['purpose_category'] = data['purpose'].apply(purpose_group) # с помощью функции создаем новый столбец с классификацией


In [22]:
data['purpose_category'].value_counts() # итак, насколько равномерно распределились наши выделенные группы по целям кредита?

операции с недвижимостью    10840
операции с автомобилем       4315
получение образования        4022
проведение свадьбы           2348
Name: purpose_category, dtype: int64

In [23]:
data.duplicated().sum() # а теперь проверим наличие явных дубликатов в таблице

71

В датафрейме обнаружилась 71 повторяющаяся строка. Для таблицы в 21000 строк это небольшой показатель, причиной возникновения может быть как случайное повторное заполнение данных пользователем, так и неправильное соединение данных из разных источников. Скорее здесь виноват человеческий фактор.

In [24]:
data = data.drop_duplicates().reset_index(drop=True) # удаляем лишние дубликаты
data.duplicated().sum() # и сразу самопроверка - лишние строки устранены

0

In [25]:
data.info()
data.describe() # просмотр информации после предобработки

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21454 non-null  int64 
 1   dob_years         21454 non-null  int64 
 2   education         21454 non-null  object
 3   education_id      21454 non-null  int64 
 4   family_status     21454 non-null  object
 5   family_status_id  21454 non-null  int64 
 6   gender            21454 non-null  object
 7   income_type       21454 non-null  object
 8   debt              21454 non-null  int64 
 9   purpose           21454 non-null  object
 10  years_employed    21454 non-null  int64 
 11  month_income      21454 non-null  int64 
 12  purpose_category  21454 non-null  object
dtypes: int64(7), object(6)
memory usage: 2.1+ MB


,children,dob_years,education_id,family_status_id,debt,years_employed,month_income
count,21454.000000,21454.000000,21454.000000,21454.000000,21454.000000,21454.000000,2.145400e+04
mean,0.480563,43.271231,0.817097,0.973898,0.081150,183.238790,1.653196e+05
std,0.756069,12.570822,0.548674,1.421567,0.273072,381.359969,9.818730e+04
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.066700e+04
25%,0.000000,33.000000,1.000000,0.000000,0.000000,2.000000,1.076230e+05
50%,0.000000,42.000000,1.000000,0.000000,0.000000,5.000000,1.425940e+05
75%,1.000000,53.000000,1.000000,1.000000,0.000000,14.000000,1.958202e+05
max,5.000000,75.000000,4.000000,4.000000,1.000000,1100.000000,2.265604e+06


Итак, пропуски устранили, дубликаты удалены, нужные столбцы приведены к типу int и выглядят опрятно. Неявные дубликаты и аномалии вроде "отрицательного ребенка" исправлены. Можно двигаться дальше.

## Шаг 4. Анализ данных

Прежде создадим словари  с данными о семейном положении и об образовании: таким образом мы разгрузим основной датафрейм и оптимизируем данные, а в будущем можно  будет обратиться к значению "словаря" по его идентификатору.

In [26]:
# создание словарей education_dict, family_dict, удаление дубликатов и создание новых индексов без сохранения старых

education_dict = (data[['education', 'education_id']]).drop_duplicates().reset_index(drop=True)
family_dict = (data[['family_status_id', 'family_status']]).drop_duplicates().reset_index(drop=True)

In [27]:
# удаление лишних столбцов 'education' и 'family_status': данные о них теперь хранятся в словарях
data = data.drop(columns=['education', 'family_status'], axis=1)

Далее распределим данные о доходах по категориям, в зависимости от показателя клиенты будут распределены по определенным группам. Категоризация поможет в дальнейшем ответить на вопрос о зависимости между уровнем зарплаты и кредитными выплатами. Напишем функцию income_group, которая присвоит каждому клиенту свою категорию соответственно его доходу.

In [28]:
def income_group(income):         # функция income_group, которая в зависимости от значения переменной income 
    if 0 <= income <= 30000:      # возвращает категорию доходности от 'E' до 'A'.
        return 'E'
    if 30001 <= income <= 50000:
        return 'D'
    if 50001 <= income <= 200000:
        return 'C'
    if 200001 <= income <= 1000000:
        return 'B'
    return 'A'

In [29]:
# создание нового столбца 'total_income_category' с присвоением каждому клиенту категории доходности
data['total_income_category'] = data['month_income'].apply(income_group)

display(data.sample(5)) # проверка функции: вывод 5 случаных строк из датафрейма

,children,dob_years,education_id,family_status_id,gender,income_type,debt,purpose,years_employed,month_income,purpose_category,total_income_category
4786,0,54,1,4,F,сотрудник,1,покупка жилья для сдачи,4,142594,операции с недвижимостью,C
9200,0,49,1,0,M,сотрудник,0,заняться высшим образованием,7,70456,получение образования,C
9883,0,57,1,1,F,госслужащий,0,на проведение свадьбы,16,131843,проведение свадьбы,C
7252,0,61,1,0,M,пенсионер,0,покупка жилья для семьи,1010,77015,операции с недвижимостью,C
1106,0,21,2,1,F,сотрудник,0,на проведение свадьбы,1,197191,проведение свадьбы,C


In [30]:
data['total_income_category'].value_counts() # как распределились доходы в наших данных

C    16015
B     5042
D      350
A       25
E       22
Name: total_income_category, dtype: int64

### Шаг 4.1. Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [31]:
# формируем сводную таблицу 
# создадим функцию для формирования сводной таблицы методом pivot_table, 
# обозначим необходимые переменные, посчитаем количество задолженностей, 
# выведем отдельный столбец ratio с отношением просрочек к выплатам

def data_pivot_table(index, values):
    pivot_table = data.pivot_table(index=index, columns='debt', values=values, aggfunc='count')
    pivot_table['ratio'] = pivot_table[1] / (pivot_table[0] + pivot_table[1]) * 100
    return pivot_table.sort_values(by=['ratio'], ascending=False)

data_pivot_table('children', 'purpose')

debt,0,1,ratio
children,,,
4,37.0,4.0,9.756098
2,1926.0,202.0,9.492481
1,4410.0,445.0,9.165808
3,303.0,27.0,8.181818
0,13028.0,1063.0,7.543822
5,9.0,NaN,NaN


Вот и первые результаты: оказывается, бездетные клиенты справляются с погашением кредита лучше, чем клиенты с детьми! Самый большое отношение просрочек у многодетный семей, но там и выборка всего 37 показателей. У клиентов с пятью детьми данных о задолженностей нет, опять же, выборка минимальна.

## Шаг 4.2. Есть ли зависимость между семейным положением и возвратом кредита в срок?


In [32]:
# используя нашу функцию, создадим сводную таблицу для показателя семейного статуса
data_family = data_pivot_table('family_status_id', 'purpose') 

# чтобы "расшифровать" столбец 'family_status_id',
# вытянем информацию из нашего словаря и добавим новый столбец, используя метод merge()
data_family.merge(family_dict, on='family_status_id', how='left')

,family_status_id,0,1,ratio,family_status
0,4,2536,274,9.750890,Не женат / не замужем
1,1,3763,388,9.347145,гражданский брак
2,0,11408,931,7.545182,женат / замужем
3,3,1110,85,7.112971,в разводе
4,2,896,63,6.569343,вдовец / вдова


Интересный результат: больше всего задолженностей у одиноких клиентов или у тех, кто не оформил официальные отношения. Любопытно, видимо, люди с узаконенным браком, или с воспоминаниями о нем, более ответственны. 
Либо молодые незамужние и неженатые еще нетвердо стоят на ногах, и у них доход непостоянный, в то время как разведенные, вдовцы, или женатые люди уже старше, имеют более стабильный доход и опыт за плечами.

## Шаг 4.3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [33]:
data_pivot_table('total_income_category', 'purpose') # сводная таблица в соответствии с уровнем дохода клиентов

debt,0,1,ratio
total_income_category,,,
E,20,2,9.090909
C,14655,1360,8.492039
A,23,2,8.000000
B,4686,356,7.060690
D,329,21,6.000000


Как это ни странно, меньше всего просрочек у людей среднего уровня дохода, богатые клиенты больше склонны задерживать выплаты. Опять же, можно пуститься в размышления, почему результат именно такой, клиенты среднего звена обычно формируют бюджет и строго знают, когда и сколько они должны выплатить банку. Богатые люди меньше переживают о выплатах, они знают, что в критической ситуации они всегда могут расплатиться и выплатить банку начисленные проценты.

 ## Шаг 4.4 Как разные цели кредита влияют на возврат в срок?

In [34]:
data_pivot_table('purpose_category', 'purpose') # сводная таблица в соответствией с категориями целей кредита

debt,0,1,ratio
purpose_category,,,
операции с автомобилем,3903,403,9.359034
получение образования,3643,370,9.220035
проведение свадьбы,2138,186,8.003442
операции с недвижимостью,10029,782,7.233373


Клиенты с ипотекой чаще остальных выплачивают кредит вовремя, чего не скажешь об образовании и приобретении авто, разница между ними составляет два процента. Потерять крышу над головой гораздо страшнее, чем потерять автомобиль, все таки это предмет не первой необходимости.

## Шаг 5. Интерпретация результатов и выводы.

Итак, после анализа можно с уверенностью сказать, что на выплату кредита в срок влияют как цели кредита, так и семеное положение с уровнем дохода. Бездетные семьи с официально зарегистрированным браков среднего класса имеют меньше просрочек, чем многодетные семьи и клиенты с доходом больше 200 тысяч в месяц. Кредит на покупку автомобиля или обучение так же связан с большим риском невыплат нежели оформление ипотеки. В целом разница между сравниваемыми значениями держится в пределах двух процентов. 

В соответствии с вышеприведенными сводными таблицами, для банка наиболее желателен клиент с официально зарегистрированным браком, без детей, с доходом выше среднего с запросом на получение ипотеки. Напротив, бедные клиенты с целью кредита на машину или обучение, без официального брака и многодетные - повод для банка задуматься о возможном отказе денежных средств. Стоит уделить внимание клиентам с высоким уровнем дохода, судя по данным, они также склонны к просрочкам.

Хотелось бы порекомендовать банку при будущих выгрузках данных провести некоторые улучшения:
- заранее автоматизировать проверку заполнения данных: исключить вероятность появления отрицательных дней или детей;
- ввести встроенную категоризацию клиентов: например, присваивать каждому свою категорию по уровню дохода;
- унифицировать заполнение некоторых данных, чтобы не возникало лишних дубликатов: например, уровень образования.